In [ ]:
import tensorflow
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import pandas as pd
import time
###
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend
from tensorflow.keras.applications import VGG19
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras import regularizers
from tensorflow.keras import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
###
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
###
from utils import process_mnist

# I. Benchmark with MNIST dataset

In [ ]:
X, y = fetch_openml('mnist_784', version = 1, return_X_y = True, as_frame = False)
X = process_mnist.resize_mnist(X, 32, 28)
###
X = np.stack((X,) * 3, axis = -1)
y = to_categorical(y, num_classes = 10)

In [ ]:
assert X.shape == (70000, 32, 32, 3), "X should have a dimension of (70000, 32, 32, 3)"
assert y.shape == (70000,10), "y should have a dimension of (70000,10)"

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = None, test_size = 10000)

In [ ]:
backend.clear_session()
try:
    os.mkdir('classifiers')
except:
    pass
full_name = 'Model A'
###
convolution_base = VGG19(weights = 'imagenet', include_top = False, 
                         input_shape = (32,32,3))
convolution_base.trainable = False
###
model = Sequential()
model.add(convolution_base)
x = model.output
###
x = Conv2D(128, kernel_size=(3,3), activation = 'relu', padding = 'same')(x)
x = Conv2D(128, kernel_size=(3,3), activation = 'relu', padding = 'same')(x)
x = Conv2D(128, kernel_size=(3,3), activation = 'relu', padding = 'same')(x)
x = Conv2D(128, kernel_size=(3,3), activation = 'relu', padding = 'same')(x)
x = GlobalAveragePooling2D()(x)
###
x = Dense(4096, activation = 'relu', kernel_regularizer = regularizers.l1_l2(l1 = 1e-5, l2 = 1e-4),
          bias_regularizer = regularizers.l2(1e-4),
          activity_regularizer = regularizers.l2(1e-5))(x)
x = Dropout(0.5)(x)
x = Dense(2048, activation = 'relu', kernel_regularizer = regularizers.l1_l2(l1 = 1e-5, l2 = 1e-4),
          bias_regularizer = regularizers.l2(1e-4),
          activity_regularizer = regularizers.l2(1e-5))(x)
x = Dropout(0.5)(x)
x = Dense(128, activation = 'relu', kernel_regularizer = regularizers.l1_l2(l1 = 1e-5, l2 = 1e-4),
          bias_regularizer = regularizers.l2(1e-4),
          activity_regularizer = regularizers.l2(1e-5))(x)
out = Dense(10, activation = 'softmax')(x)
finalModel = Model(inputs = model.input, outputs = out)
###
finalModel.summary()
###
opt = Adam(learning_rate = 0.001, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-07)
finalModel.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy', 'Precision', 'Recall'])
###
#filepath = "classifiers/%s-{epoch:02d}-{val_accuracy:.4f}-MNIST.hdf5"%full_name
checkpoint = ModelCheckpoint(filepath, monitor = 'val_accuracy', save_best_only = True, mode = 'max')
callbacks_list = [checkpoint]

start = time.time()
history = finalModel.fit(X_train, y_train, epochs = 100, verbose = 1, callbacks = callback_list,
                         shuffle = True, validation_data = (X_test, y_test))
end = time.time()
runtime_mnist = end - start
###
print(f'Total runtime: {round(runtime_mnist, 2)}')

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Model A's Accuracy")
plt.ylabel('Accuracy (%)')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc = 'upper left')
plt.show()
###
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Model A's Accuracy")
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc = 'upper left')
plt.grid(True)
plt.show()

## II. Benckmark with Fashion-MNIST dataset:

In [ ]:
X_train, y_train = process_mnist.load_mnist('data/fashion', kind = 'train')
X_test, y_test = process_mnist.load_mnist('data/fashion', kind = 't10k')
###
X_train = process_mnist.resize_mnist(X_train, 32, 28)
X_train = np.stack((X_train,) * 3, axis = -1)
X_test = process_mnist.resize_mnist(X_test, 32, 28)
X_test = np.stack((X_test,) * 3, axis = -1)
###
y_train = to_categorical(y_train, num_classes = 10)
y_test = to_categorical(y_test, num_classes = 10)

In [ ]:
assert X_train.shape == (60000,32,32,3), "X_train should have a dimension of (60000,32,32,3)!"
assert X_test.shape == (10000,32,32,3), "X_test should have a dimension of (10000,32,32,3)!"
assert y_train.shape == (60000,10) "y_train should have a dimension of (60000,10)"
assert y_test.shape == (10000,10) "y_test should have a dimension of (10000,10)"

In [ ]:
start = time.time()
history = finalModel.fit(X_train, y_train, epochs = 100, verbose = 1, callbacks = callback_list,
                         shuffle = True, validation_data = (X_test, y_test))
end = time.time()
runtime_mnist = end - start
###
print(f'Total runtime: {round(runtime_mnist, 2)}')

In [ ]:
###
#filepath = "classifiers/%s-{epoch:02d}-{val_accuracy:.4f}-fashion.hdf5"%full_name
checkpoint = ModelCheckpoint(filepath, monitor = 'val_accuracy', save_best_only = True, mode = 'max')
callbacks_list = [checkpoint]

In [ ]:
start = time.time()
history = finalModel.fit(X_train, y_train, epochs = 100, verbose = 1,
                         shuffle = True, validation_data = (X_test, y_test))
end = time.time()
runtime_fashion = end - start
###
print(f'Total runtime: {round(runtime_fashion, 2)}')